In [ ]:
# 중학 수학2 영상 저장
# 영상 출력 없이 결과값만 저장 / 3초에 1번씩 프레임 처리

import os
import glob
import json
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

# 감정 레이블과 색상 정보 저장
emotions = {
    0: ['Angry', (0, 0, 255), (255, 255, 255)],
    1: ['Disgust', (0, 102, 0), (255, 255, 255)],
    2: ['Fear', (255, 255, 153), (0, 51, 51)],
    3: ['Happy', (153, 0, 153), (255, 255, 255)],
    4: ['Sad', (255, 0, 0), (255, 255, 255)],
    5: ['Surprise', (0, 255, 0), (255, 255, 255)],
    6: ['Neutral', (160, 160, 160), (255, 255, 255)]
}

num_classes = len(emotions) # 감정 클래스 수
input_shape = (48, 48, 1) # 모델의 입력 이미지 형태
weights_1 = 'vggnet.h5' # 감정 분류 모델
weights_2 = 'vggnet_up.h5' # 감정 분류 가중치 모델


# VGGNet 클래스 정의 - Keras의 Sequential 클래스를 상속하여 신경망의 레이어를 선형으로 쌓는다.
class VGGNet(Sequential):
    # 클래스 생성자 메서드. 클래스의 인스턴스가 생성될 때 객체 초기화
    def __init__(self, input_shape, num_classes, checkpoint_path, lr=1e-3):
        # input_shape: 입력 데이터의 형태(이미지 크기)
        # num_classes: 분류 작업의 클래스 수
        # checkpoint_path: 모델 가중치를 저장하거나 불러올 경로
        # lr=1e-3: 학습률, 기본값은 0.001
        super().__init__()
        self.add(Rescaling(1. / 255, input_shape=input_shape)) # 입력 픽셀 값을 [0, 1] 범위로 재조정
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
        self.add(BatchNormalization())
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5)) # 훈련 중 과적합을 방지하기 위해 Dropout 레이어가 추가

        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Flatten()) # 3D 출력을 1D로 변환하기 위해 Flatten 레이어를 추가
        
        # ReLU 활성화 함수를 사용한 두 개의 완전 연결 Dense 레이어가 추가
        self.add(Dense(1024, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(256, activation='relu'))

        #소프트맥스 활성화 함수를 사용한 마지막 Dense 레이어를 추가
        # 이 레이어는 각 클래스에 대한 출력 확률을 생성
        self.add(Dense(num_classes, activation='softmax')) # 

        # 모델을 Adam 옵티마이저, categorical crossentropy 손실 함수 및 정확도 지표와 함께 컴파일함
        self.compile(optimizer=Adam(learning_rate=lr),
                     loss=categorical_crossentropy,
                     metrics=['accuracy'])

        # 나중에 사용하기 위해 제공된 checkpoint_path를 저장
        self.checkpoint_path = checkpoint_path

# VGGNet 모델의 인스턴스를 생성하고 미리 학습된 가중치를 로드
model_1 = VGGNet(input_shape, num_classes, weights_1)
model_1.load_weights(model_1.checkpoint_path)

model_2 = VGGNet(input_shape, num_classes, weights_2)
model_2.load_weights(model_2.checkpoint_path)

# 미디어파이프를 사용하여 얼굴 감지를 위한 모듈과 객체 초기화
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# 이미지 크기를 조정하여 얼굴 감지를 수행하기 전에 전처리
# 이미지의 높이를 최대 높이로 제한하고 비율을 유지한 채로 이미지를 크기 조정
def detection_preprocessing(image, h_max=360):
    h, w, _ = image.shape
    if h > h_max:
        ratio = h_max / h
        w_ = int(w * ratio)
        image = cv2.resize(image, (w_, h_max))
    return image

# 얼굴 이미지를 크기 조정하고 모델 입력에 맞게 전처리
# 감정 분류 모델에 입력으로 주어질 이미지들을 사전 처리하기 위한 것으로, 이미지 크기 조정과 텐서 형태로의 변환을 수행
def resize_face(face):
    x = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
    return tf.image.resize(x, (48, 48))
def recognition_preprocessing(faces):
    x = tf.convert_to_tensor([resize_face(f) for f in faces])
    return x

# 이미지에서 얼굴을 감지하고 감정을 추론
def inference(image):

    global emotion_counts # 감정 클래스별로 이미지에서 감지된 얼굴의 수 기록
    H, W, _ = image.shape # 입력 이미지의 높이와 너비를 가져온다.

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지의 색상 채널을 BGR에서 RGB로 변환
    results = face_detection.process(rgb_image)

    # 얼굴이 감지된 경우에만 아래 코드 실행
    if results.detections:
        # 각 얼굴에 대한 감지 결과를 순회
        for detection in results.detections:
            # 얼굴의 경계 상자 정보를 가져온다.
            box = detection.location_data.relative_bounding_box
            # 경계 상자의 좌표와 크기 정보를 계산
            x = int(box.xmin * W)
            y = int(box.ymin * H)
            w = int(box.width * W)
            h = int(box.height * H)

            # 경계 상자의 좌표를 조정하여 이미지에서 얼굴 부분을 잘라낸다.
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(x + w, W)
            y2 = min(y + h, H)

            # 원본 이미지에서 얼굴 부분을 추출
            face = image[y1:y2, x1:x2]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) # 추출된 얼굴 이미지를 흑백 이미지로 변환

            face = tf.expand_dims(tf.convert_to_tensor(face), axis=2) # TensorFlow의 텐서 형태로 변환하고, 차원을 추가하여 모델에 입력으로 사용
            face = tf.image.resize(face, (48, 48)) # 얼굴 이미지를 감정 분류 모델에 맞는 크기로 조정

            
            x = recognition_preprocessing(face)
            y_1 = model_1.predict(x) # 첫 번째 모델로 감정을 예측
            y_2 = model_2.predict(x) # 두 번째 모델로 감정을 예측
            l = np.argmax(y_1 + y_2, axis=1) # 두 모델의 예측 결과를 합산하여 가장 확률이 높은 감정 클래스를 선택
            emotion_counts[l[0]] += 1


# Video 감정분석
def save_emotion_results(video_path, emotion_counts):
    # 각 동영상에 대한 감정 분석 결과를 저장할 딕셔너리 생성
    emotion_results = {
        "emotion_counts": emotion_counts,
        "emotion_names": [emotions[label][0] for label in emotion_counts.keys()],
        "emotion_values": list(emotion_counts.values())
    }

    # 개별 동영상 결과를 저장
    json_file_path = f'emotion_results_{os.path.basename(video_path)}.json'
    json_file_path = os.path.join(video_folder_path, json_file_path)

    with open(json_file_path, 'w') as json_file:
        json.dump(emotion_results, json_file, indent=4)

    return emotion_results


# 모든 동영상 결과를 저장할 빈 딕셔너리 초기화
combined_results = {}

# 동영상 파일이 있는 폴더 경로
video_folder_path = 'C:/Users/user/Downloads/media'

# 폴더 내의 모든 동영상 파일 경로를 얻어옴
video_paths = glob.glob(os.path.join(video_folder_path, '*.mp4'))

# 각 동영상에 대한 반복문을 시작
for video_path in video_paths:
    cap = cv2.VideoCapture(video_path) # 동영상 파일을 열고 객체 생성

    # 동영상의 총 프레임 수, 프레임 속도, 동영상 길이를 가져오기
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_duration = total_frames / fps

    # 감정별 카운트 초기화 (각 프레임당 한 번 초기화)
    emotion_counts = {emotion_label: 0 for emotion_label in emotions}

    # 3초당 한 번씩 프레임을 처리하기 위한 카운터 초기화
    idx = 0

    start_time = time.time()
    
    # 동영상의 모든 프레임에 대한 무한 루프 시작
    while True:
        success, image = cap.read() # 동영상에서 한 프레임을 읽기
        if not success:
            break
        
        # 현재 시간과 경과 시간을 측정
        current_time = time.time()
        elapsed_time = current_time - start_time

        # 3초당 1번씩 처리 (간격을 더 짧게 조정하여 자주 감정 분석)
        # 현재 시간을 초로 변환
        seconds = np.floor(cap.get(cv2.CAP_PROP_POS_MSEC)/ 1000) * 1000 - idx
        if seconds % 3 == 0:
            idx += 3
            # 얼굴을 감지하고 감정을 추론
            inference(image)  # 'inference' 함수에서 얼굴을 가져옵니다

            # 초기화 및 최종 결과 저장
            # 결과 저장하는 시간이 처리하는 시간이랑 똑같다.
            # if elapsed_time >= 3.0:
            save_emotion_results(video_path, emotion_counts)
            start_time = time.time()

    cap.release() # 동영상 파일을 닫고 객체 해제

    # 통합된 딕셔너리에 결과 저장
    combined_results[os.path.basename(video_path)] = emotion_counts

# 통합된 결과를 하나의 JSON 파일로 저장
combined_json_file_path = os.path.join(video_folder_path, 'combined_emotion_results.json')
with open(combined_json_file_path, 'w') as json_file:
    json.dump(combined_results, json_file, indent=4)

print(f"통합된 감정 분석 결과가 {combined_json_file_path}에 저장되었습니다.")

---- ㅈㅎㅅ 감정 분석 코드 구현 ----

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import os
from collections import OrderedDict
# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# 감정 분석 모델 로드
emotion_model = load_model('./emotion_model.h5')
# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
# 동영상 파일이 있는 폴더 경로
folder_path = 'D:\EBS중학뉴런수학2(상)_손석민'  
# 결과를 저장할 JSON 파일 경로
output_json_path = 'data_Emotional.json'
# 결과를 저장할 OrderedDict 초기화
emotion_data = OrderedDict()
emotion_data['data'] = []
# 폴더 안에 있는 동영상 파일들을 대상으로 처리
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp4'):
        video_path = os.path.join(folder_path, file_name)
        # 동영상 파일 열기
        cap = cv2.VideoCapture(video_path)
        # 감정별 횟수 카운트 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}
        # 감정 수행 간격을 확인하기 위한 카운트 초기화
        frame_count = 0
        while True:
            # 동영상에서 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break
            # 그레이스케일로 변환
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # 얼굴 감지
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))
            # 얼굴이 하나 이상 감지되었을 때만 처리
            if len(faces) > 0:
                # 가장 첫 번째 얼굴에 대해서만 처리
                x, y, w, h = faces[0]
                face = gray[y:y + h, x:x + w]
                face = cv2.resize(face, (48, 48))
                face = face / 255.0  # 이미지 정규화
                # 5프레임마다 감정 분석 수행
                if frame_count % 5 == 0:
                    # 모델 입력 형식으로 변환
                    face = tf.expand_dims(face, axis=-1)
                    face = tf.expand_dims(face, axis=0)
                    # 감정 분석 수행
                    emotion_prediction = emotion_model.predict(face)
                    emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]
                    # 감정 카운트 증가
                    emotion_counts[emotion_label] += 1
            frame_count += 1  # 프레임 카운트 증가
        # 리소스 해제
        cap.release()
        # 결과를 OrderedDict에 추가
        emotion_data['data'].append({video_path: emotion_counts})
# OrderedDict를 JSON 파일로 저장
with open(output_json_path, 'w') as f:
    json.dump(emotion_data, f, ensure_ascii=False, indent=4)

In [ ]:
# ㅈㅎㅅ 코드 수정

import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import os
from collections import OrderedDict
# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# 감정 분석 모델 로드
emotion_model = load_model('../emotion_model.h5')
# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
# 동영상 파일이 있는 폴더 경로
folder_path = 'D:\ebs\변창현'
# folder_path = 'D:\ebs\손석민(0-9)'
# 결과를 저장할 JSON 파일 경로
output_json_path = 'data_Emotional_1.json'
# 결과를 저장할 OrderedDict 초기화
emotion_data = OrderedDict()
emotion_data['data'] = []
# 폴더 안에 있는 동영상 파일들을 대상으로 처리
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp4'):
        video_path = os.path.join(folder_path, file_name)
        # 동영상 파일 열기
        cap = cv2.VideoCapture(video_path)
        # 감정별 횟수 카운트 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}
        # 감정 수행 간격을 확인하기 위한 카운트 초기화
        frame_count = 0
        while True:
            # 동영상에서 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break
            # 그레이스케일로 변환
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # 얼굴 감지
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))
            # 얼굴이 하나 이상 감지되었을 때만 처리
            if len(faces) > 0:
                # 가장 첫 번째 얼굴에 대해서만 처리
                x, y, w, h = faces[0]
                face = gray[y:y + h, x:x + w]
                face = cv2.resize(face, (48, 48))
                face = face / 255.0  # 이미지 정규화
                # 5프레임마다 감정 분석 수행
                if frame_count % 5 == 0:
                    # 모델 입력 형식으로 변환
                    face = tf.expand_dims(face, axis=-1)
                    face = tf.expand_dims(face, axis=0)
                    # 감정 분석 수행
                    emotion_prediction = emotion_model.predict(face)
                    emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]
                    # 감정 카운트 증가
                    emotion_counts[emotion_label] += 1
            frame_count += 1  # 프레임 카운트 증가
        # 리소스 해제
        cap.release()
        # 결과를 OrderedDict에 추가
        emotion_data['data'].append({video_path: emotion_counts})
    # OrderedDict를 JSON 파일로 저장
    with open(output_json_path, 'w') as f:
        json.dump(emotion_data, f, ensure_ascii=False, indent=4)

-- dash로 그래프 만드는 코드 --

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import boto3
from botocore.exceptions import NoCredentialsError
from io import BytesIO

# AWS S3 관련 설정 - 변경 필요
AWS_ACCESS_KEY = 'your_access_key'
AWS_SECRET_KEY = 'your_secret_key'
S3_BUCKET = 'big-project-team3-emotion'
S3_REGION = 'ap-northeast-2'

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 설정
app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            '드래그 앤 드롭 또는 ',
            html.A('파일 선택')
        ]),
        multiple=False
    ),
    html.Div(id='output-data-upload')
])

# 파일 업로드 콜백 함수
@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'))
def update_output(contents):
    if contents is not None:
        try:
            # AWS S3에 파일 업로드
            s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=S3_REGION)
            
            # BytesIO를 사용하여 contents를 읽어옴
            file_content = contents.encode('utf-8')
            file_stream = BytesIO(file_content)

            # S3에 업로드
            s3.upload_fileobj(file_stream, S3_BUCKET, 'uploaded_video.mp4')

            return html.Div([
                '파일이 성공적으로 업로드되었습니다.',
                html.Video(src=f"https://{S3_BUCKET}.s3.{S3_REGION}.amazonaws.com/uploaded_video.mp4", controls=True)
            ])
        except NoCredentialsError:
            return 'AWS 자격 증명이 올바르지 않습니다.'
        except Exception as e:
            return f'에러 발생: {str(e)}'

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)

# 127.0.0.1:8050

In [39]:
# 만들어진 json 파일을 하나의 json 파일로 통합하는 코드

import os
import json
import glob

emotions = {
    0: ['Angry', (0, 0, 255), (255, 255, 255)],
    1: ['Disgust', (0, 102, 0), (255, 255, 255)],
    2: ['Fear', (255, 255, 153), (0, 51, 51)],
    3: ['Happy', (153, 0, 153), (255, 255, 255)],
    4: ['Sad', (255, 0, 0), (255, 255, 255)],
    5: ['Surprise', (0, 255, 0), (255, 255, 255)],
    6: ['Neutral', (160, 160, 160), (255, 255, 255)]
}

# 통합될 JSON 파일의 경로
json_folder_path = 'C:/Users/user/Downloads/감정분석 결과/EBS중학뉴런수학2(상)_손석민_감정분석'

# 통합 결과를 저장할 빈 딕셔너리 초기화
consolidated_results = {
    "emotion_counts": {emotion_label: 0 for emotion_label in emotions},
    "emotion_names": [emotions[label][0] for label in emotions],
    "emotion_values": []
}

# JSON 파일 목록 얻기
json_files = [f for f in os.listdir(json_folder_path) if f.endswith('.json')]

# 각 JSON 파일 읽어와서 통합
for json_file in json_files:
    with open(os.path.join(json_folder_path, json_file), 'r') as file:
        data = json.load(file)
        consolidated_results["emotion_values"].append(data["emotion_counts"])

        # 감정 카운트 집계
        for label, count in data["emotion_counts"].items():
            consolidated_results["emotion_counts"][int(label)] += count

# 통합된 JSON 파일 저장
consolidated_json_path = os.path.join(json_folder_path, 'consolidated_results.json')
with open(consolidated_json_path, 'w') as consolidated_file:
    json.dump(consolidated_results, consolidated_file, indent=4)

print(f'The consolidated results are saved to {consolidated_json_path}')

The consolidated results are saved to C:/Users/user/Downloads/감정분석 결과/EBS중학뉴런수학2(상)_손석민_감정분석\consolidated_results.json


In [1]:
import os
import json
import plotly.express as px

emotions = {
    0: ['Angry', (0, 0, 255), (255, 255, 255)],
    1: ['Disgust', (0, 102, 0), (255, 255, 255)],
    2: ['Fear', (255, 255, 153), (0, 51, 51)],
    3: ['Happy', (153, 0, 153), (255, 255, 255)],
    4: ['Sad', (255, 0, 0), (255, 255, 255)],
    5: ['Surprise', (0, 255, 0), (255, 255, 255)],
    6: ['Neutral', (160, 160, 160), (255, 255, 255)]
}

# JSON 파일이 있는 폴더 경로
json_folder_path = 'C:/Users/user/Downloads/media'

# 통합 결과를 저장할 빈 딕셔너리 초기화
all_consolidated_results = {}

# JSON 파일 목록 얻기
json_files = [f for f in os.listdir(json_folder_path) if f.endswith('.json')]

# 각 JSON 파일 읽어와서 통합
for json_file in json_files:
    with open(os.path.join(json_folder_path, json_file), 'r') as file:
        data = json.load(file)
        # 파일 이름을 통한 구분 (예: 파일명이 'result1.json'이면 'result1'이 키로 사용됨)
        key = os.path.splitext(json_file)[0]
        all_consolidated_results[key] = {
            "emotion_counts": {emotion_label: 0 for emotion_label in emotions},
            "emotion_names": [emotions[label][0] for label in emotions],
            "emotion_values": data["emotion_counts"]
        }

        # 감정 카운트 집계
        for label, count in data["emotion_counts"].items():
            all_consolidated_results[key]["emotion_counts"][int(label)] += count

# 모든 통합 결과에 대해 그래프 생성
for key, consolidated_results in all_consolidated_results.items():
    fig = px.bar(
        x=consolidated_results['emotion_names'],
        y=consolidated_results['emotion_counts'],
        labels={'x': 'Emotion', 'y': 'Count'},
        title=f'Emotion Analysis Results - {key}',
        color=consolidated_results['emotion_names']  # 각각의 컬럼에 대해 다른 색상 사용
    )

    # 여기서는 그래프를 화면에 표시하지만, 원하는 방식으로 활용 가능
    fig.show()

In [ ]:
# 코드 확장
# s3에 영상 적재하면 영상에서 데이터 추출을 ecs 나 fargate에서 하고,
# 그 추출된 데이터를 rds에 저장하고 
# 저장된 데이터를 ec2에 가져오는 과정에서 sqs를 사용하고 가져온 데이터를 dash에 보여주기

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import boto3
from botocore.exceptions import NoCredentialsError
from io import BytesIO

# AWS S3, SQS, RDS 관련 설정 - 변경 필요
AWS_ACCESS_KEY = 'your_access_key'
AWS_SECRET_KEY = 'your_secret_key'
S3_BUCKET = 'big-project-team3-emotion'
S3_REGION = 'ap-northeast-2'
SQS_QUEUE_URL = 'your_sqs_queue_url'
RDS_ENDPOINT = 'your_rds_endpoint'
RDS_PORT = 'your_rds_port'
RDS_USER = 'your_rds_user'
RDS_PASSWORD = 'your_rds_password'
RDS_DB_NAME = 'your_rds_db_name'

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 설정
app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            '드래그 앤 드롭 또는 ',
            html.A('파일 선택')
        ]),
        multiple=False
    ),
    html.Div(id='output-data-upload')
])

# 파일 업로드 콜백 함수
@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'))
def update_output(contents):
    if contents is not None:
        try:
            # AWS S3에 파일 업로드
            s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=S3_REGION)
            file_content = contents.encode('utf-8')
            file_stream = BytesIO(file_content)
            s3.upload_fileobj(file_stream, S3_BUCKET, 'uploaded_video.mp4')

            # SQS에 메시지 전송
            sqs = boto3.client('sqs', region_name=S3_REGION)
            sqs.send_message(QueueUrl=SQS_QUEUE_URL, MessageBody='{"data_location": "s3://{}/uploaded_video.mp4"}')

            return html.Div([
                '파일이 성공적으로 업로드되었습니다.',
                html.Video(src=f"https://{S3_BUCKET}.s3.{S3_REGION}.amazonaws.com/uploaded_video.mp4", controls=True)
            ])
        except NoCredentialsError:
            return 'AWS 자격 증명이 올바르지 않습니다.'
        except Exception as e:
            return f'에러 발생: {str(e)}'

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)



# 이 코드에서는 업로드된 파일이 S3에 업로드되면, SQS에 메시지가 전송되고 
# 해당 메시지에는 추출된 데이터의 위치인 S3 경로가 포함됩니다. 
# 이 메시지는 다른 서비스가 해당 데이터를 가져와서 필요한 처리를 수행하도록 도와줍니다. 
# 필요에 따라 ECS, Fargate, RDS에서의 데이터 처리 부분을 추가로 구현해야 합니다.